In [ ]:
# MWE of ContinuousCallback + SavingCallback

In [6]:
using DiffEqBase, OrdinaryDiffEq, DiffEqCallbacks, DataFrames

### ODE
function ode_(du, u, p, t)
    (C,S2,k1,k2) = p
    (S1,S3) = u 

    reaction1 = C * k1 * S1 * S2
    reaction2 = C * k2 * S3

    du .= [
      -reaction1+reaction2,  # dS1/dt
      reaction1-reaction2,  # dS3/dt
    ]
end

### events
event1_condition_(u, t, integrator) = 0.75 - u[1]

function event1_assignment_(integrator)
    # check if saveat points are present before applying the event
    save_func! = integrator.opts.callback.discrete_callbacks[1].affect!
    save_func!(integrator)
    
    # save point before event
    save_func!(integrator, true)
    
    #event
    integrator.p[2] = 1.0
    
    # save point after event
    save_func!(integrator, true)
end

event2_condition_(u, t, integrator) = u[2] - 1.4

function event2_assignment_(integrator)
    # check if saveat points are present before applying the event
    save_func! = integrator.opts.callback.discrete_callbacks[1].affect!
    save_func!(integrator)
        
    # save point before event
    save_func!(integrator, true)
    
    # event
    integrator.u[1] = 1.0
    
    # save point after event
    save_func!(integrator, true)
end

event1 = ContinuousCallback(event1_condition_, event1_assignment_,
        save_positions=(false,false)
)

event2 = ContinuousCallback(event2_condition_, event2_assignment_,
        save_positions=(false,false)
);

In [3]:
# solver options
tspan = (0.0, 3.0)
step = 0.06
saveat = collect(range(tspan[1],tspan[2], step=step))
abstol = 1e-3
reltol = 1e-3

# initial values and parameters
u0 = [1.0, 1.0] # S1, S3
p0 = [1.0, 2.0, 0.75, 0.25] # C, S2, k1, k2


### output function
saving_(u, t, integrator) = [u[1], integrator.p[2], u[2]] # S1, S2, S3
out = SavedValues(Float64, Vector{Float64})
scb = SavingCallback(saving_, out, saveat=saveat)

### Callback set
cbset = CallbackSet(
    scb, 
    event1, 
    event2,
)

# ODE Problem
prob = ODEProblem(ode_, u0, tspan, p0, callback=cbset)

ODEProblem with uType Array{Float64,1} and tType Float64. In-place: true
timespan: (0.0, 3.0)
u0: [1.0, 1.0]

In [4]:
#solving the problem
sol = solve(prob, Vern9(), reltol=reltol, abstol=abstol,
        save_start = false,
        save_end = false,
        save_everystep = false,
)

retcode: Success
Interpolation: 1st order linear
t: 0-element Array{Float64,1}
u: 0-element Array{Array{Float64,1},1}

In [5]:
# print results
DataFrame(
    t=out.t, 
    S1=[sv[1] for sv in out.saveval], 
    S2=[sv[2] for sv in out.saveval], 
    S3=[sv[3] for sv in out.saveval]
)

,t,S1,S2,S3
,Float64,Float64,Float64,Float64
1,0.0,1.0,2.0,1.0
2,0.06,0.928803,2.0,1.0712
3,0.12,0.864703,2.0,1.1353
4,0.18,0.806992,2.0,1.19301
5,0.24,0.755033,2.0,1.24497
6,0.3,0.708254,2.0,1.29175
7,0.36,0.666137,2.0,1.33386
8,0.388549,0.6,2.0,1.4
9,0.388549,1.0,2.0,1.4


In [ ]:
# Here the second event happened at t = 1.0648, which is wrong
# The correct answer (t = 1.16) can be obtained by limiting dtmax

In [26]:
dtmax = step/3

# initial values and parameters
u0 = [1.0, 1.0] # S1, S3
p0 = [1.0, 2.0, 0.75, 0.25] # C, S2, k1, k2

# ODE Problem
prob = ODEProblem(ode_, u0, tspan, p0, callback=cbset)
sol = solve(prob, Vern9(), reltol=reltol, abstol=abstol,
        dtmax=dtmax,
        save_start = false,
        save_end = false,
        save_everystep = false,
)

retcode: Success
Interpolation: 1st order linear
t: 0-element Array{Float64,1}
u: 0-element Array{Array{Float64,1},1}

In [27]:
# print results
DataFrame(
    t=out.t, 
    S1=[sv[1] for sv in out.saveval], 
    S2=[sv[2] for sv in out.saveval], 
    S3=[sv[3] for sv in out.saveval]
)

,t,S1,S2,S3
,Float64,Float64,Float64,Float64
1,0.0,1.0,2.0,1.0
2,0.06,0.928803,2.0,1.0712
3,0.12,0.864703,2.0,1.1353
4,0.18,0.806992,2.0,1.19301
5,0.24,0.755033,2.0,1.24497
6,0.246162,0.75,2.0,1.25
7,0.246162,0.75,1.0,1.25
8,0.3,0.736896,1.0,1.2631
9,0.36,0.723101,1.0,1.2769
